Api Building


In [ ]:
import requests
import pandas as pd
import json
from datetime import datetime
import webbrowser
import time
from google.colab import drive
drive.mount('/content/drive')

historic_ether_price = pd.read_csv("/content/drive/MyDrive/Crypto/Analysis/eth-usd-max.csv") 
preprocessed_ether_address = pd.read_csv("/content/drive/MyDrive/Crypto/Analysis/Main 1to21Dec2022alladdress_morethan1txn.csv")

import sys
sys.path.insert(0,'/content/drive/MyDrive/Crypto/Analysis/ABI.py')
from ABI import ABI

Mounted at /content/drive


In [ ]:
from web3 import Web3
w3 = Web3(Web3.WebsocketProvider('wss://eth-mainnet.g.alchemy.com/v2/45gNIRViOhvLosZx4_FFdPMVvxYauBTd'))
contract = w3.eth.contract('0x7a250d5630B4cF539739dF2C5dAcb4c659F2488D', abi=ABI)
# func_obj, func_params = contract.decode_function_input(json_data['result'][0]['input'])

# method_name = str(func_obj).split(' ')[1].split('(')[0]

In [ ]:
# address_list = [x.lower() for x in list(preprocessed_ether_address['address'])[5001:]]
# Address you want the data profit and loss of :
address_list = [x.lower() for x in [
'0x78dc45c755a483e75b81da74988eb6d464c34402']]

In [ ]:
"""
EVERY ETHEREUM TXN DETAILS CAN BE FOUND WITHIN:
a) ALL TXN (TXN SEND "BY PERSON OR TO PERSON" TO THE ETHEREUM BLOCKCHAIN )
b) INTERNAL TXN (ALL ETHER TXN (ETHEREUM NATIVE CURRENCY))
c) ERC20 TXN  (ERC20 TOKEN TXN (DIVISIBLE TOKEN, COULD BE GENERATED BY ANYONE INSIDE ETHEREUM BLOCKCHAIN))
NOTE : DON'T BE CONFUSED BETWEEN WETH AND ETHER, WETH IS ERC20 TOKEN VERSION OF ETHER. 
THEREFORE, WETH CAN BE FOUND IN ERC 20 TXN AND ETHER CAN BE FOUND IN INTERNAL TXN.
THE VALUE OF WETH AND ETH IS SAME IN TERMS OF USD.
"""

In [ ]:
"""
TERMINOLOGY :
- GAS : GAS USED FOR A TXN (GAS UNITS * GAS PRICE) [THE VALUE CALCULATED THROUGHOUT IS IN TERMS OF ETHERS]
- VALUE : THE AMOUNT USED TO BUY/SELL IN TERMS OF ETHERS 
- TXN SEND BY PERSON OR RECEIVED BY THE PERSON : ALL TXN
- TRANSFER OF ETHERS FROM THE DESIRED ACCOUNT : INTERNAL TXN
- TRANSFER OF ERC20 TOKENS FROM THE DESIRED ACCOUNT : ERC20 TXN
- DESIRED ADDRESS : ETHEREUM ADDRESS ON WHICH ANALYSIS IS DONE
"""

In [ ]:
"""
NOTE : WE HAVE FILTERED UNWANTED METHODS LIKE STAKING, NFT BUY BEFORE THE FOLLOWING DETAILS, STILL WE ARE PRONE TO SOME
UNWANTED METHODS (WHICH WE WILL BE ADDED IN DOUBTFULL TXN DICTIONARY)

FLOWCHART:
WE CHECK ALL TRANSACTION IN ALL TXN DICTIONARY ONE BY ONE
CHECK ALL TXN IF IT HAS NON-ZERO VALUE (THE TXN IS FOR BUY), ALL THE ETHERS SENT IN TXN MAY NOT BE USED SO WE CHECK INTERNAL FOR ANY UNSUSED ETHERS.

"""

In [ ]:
#Retrieve ERC Transfer, Internal Txn, All Txn Data
all_transactions_of_address =[]
all_erc_20_transfers = []
all_internal_transactions = []

def update_lists(address):
  
    txlist_api = 'https://api.etherscan.io/api?module=account&action=txlist&address={}&sort=asc&apikey=JV6FAU5SY7FDUB8AMI1VHXKS6QIASF89DC'.format(address)
    all_transactions = requests.request("GET", txlist_api)
    # print("all: ",all_transactions.text[:100])
    all_transactions_of_address = json.loads(all_transactions.text)['result']

    while len(all_transactions_of_address) > 9999 and len(all_transactions_of_address) < 50000:
        txlist_api_repeat = 'https://api.etherscan.io/api?module=account&action=txlist&startblock='+ all_transactions_of_address[-1]['blockNumber'] +'&sort=asc&address={}&apikey=JV6FAU5SY7FDUB8AMI1VHXKS6QIASF89DC'.format(address)
        all_transaction_repeat = requests.request("GET", txlist_api_repeat)
        result = json.loads(all_transaction_repeat.text)['result'][1:]

        all_transactions_of_address = all_transactions_of_address + result
        if len(result) < 9999:
            break

    erc_api = 'https://api.etherscan.io/api?module=account&action=tokentx&address={}&page=1&offset=10000&sort=asc&apikey=FI33TTAM3S1WVTGG5JDDMNTI61SPQ2GTUY '.format(address)
    erc_transfers = requests.request("GET", erc_api)
    # print("erc: ",erc_transfers.text[:100])
    all_erc_20_transfers = json.loads(erc_transfers.text)['result']

    while len(all_erc_20_transfers) > 9999 and len(all_erc_20_transfers) < 50000:
        txlist_api_repeat = 'https://api.etherscan.io/api?module=account&action=tokentx&startblock='+ all_erc_20_transfers[-1]['blockNumber'] +'&sort=asc&address={}&apikey=FI33TTAM3S1WVTGG5JDDMNTI61SPQ2GTUY'.format(address)
        all_transaction_repeat = requests.request("GET", txlist_api_repeat)
        result = json.loads(all_transaction_repeat.text)['result'][1:]

        all_erc_20_transfers = all_erc_20_transfers + result
        if len(result) < 9999:
            break

    txlist_internal_api = 'https://api.etherscan.io/api?module=account&action=txlistinternal&address={}&page=1&offset=10000&sort=asc&apikey=TRT149Y52NFS2DYR6HPJF3QW2P3DBTTAJ2'.format(address)
    internal_transactions = requests.request("GET", txlist_internal_api)
    # print("internal: ",internal_transactions.text[:100])
    all_internal_transactions = json.loads(internal_transactions.text)['result']

    while len(all_internal_transactions) > 9999 and len(all_internal_transactions) < 50000:
        txlist_api_repeat = 'https://api.etherscan.io/api?module=account&action=txlistinternal&startblock='+ all_internal_transactions[-1]['blockNumber'] +'&sort=asc&address={}&apikey=TRT149Y52NFS2DYR6HPJF3QW2P3DBTTAJ2'.format(address)
        all_transaction_repeat = requests.request("GET", txlist_api_repeat)
        result = json.loads(all_transaction_repeat.text)['result'][1:]

        all_internal_transactions = all_internal_transactions + result
        if len(result) < 9999:
           break


In [ ]:
def get_method_name_and_addresses(transaction):
    try:
        func_obj, func_params = contract.decode_function_input(transaction['input'])
        return [str(transaction['functionName']).split('(')[0], func_params['path']]
    except:
        return [transaction['functionName'], []]

  # V0 version
# def get_crypto_price(symbol, start, end):
#   try:
#     api_url = f'https://data.messari.io/api/v1/markets/binance-{symbol}-usdt/metrics/price/time-series?start={start}&end={end}&interval=1d'
#     raw = requests.get(api_url).json()
#     df = pd.DataFrame(raw['data']['values'])
#     df = df.rename(columns = {0:'date',1:'open',2:'high',3:'low',4:'close',5:'volume'})
#     df['date'] = pd.to_datetime(df['date'], unit = 'ms')
#     df = df.set_index('date')
#     return df
#   except:
#     print('Taking some rest xD')
#     time.sleep(60)
#     print('Woah I woke up.!!!')
#     return get_crypto_price(symbol, start, end)

# def get_ether_price(time_stamp):
#   start = str(datetime.datetime.fromtimestamp(int(time_stamp))).split()[0]
#   end = str(datetime.datetime.fromtimestamp(int(time_stamp)) + datetime.timedelta(days=1)).split()[0]
#   eth = get_crypto_price('eth', start, end)
#   return (eth['high'][0]+eth['low'][0])/2

def get_ether_price(time_stamp):
    """
    RETURNS ETHER PRICE FOR A PARTICULAR DAY FROM TIMESTAMP, 
    IF DAILY PRICE IS NOT UPDATED, RETURNS PRICE OF LAST DAY FROM THE LIST
    """
    index = len(historic_ether_price['snapped_at'])-2
    date = str(datetime.fromtimestamp(int(time_stamp))).split()[0]
    for i,timestamp in enumerate(historic_ether_price['snapped_at']):
        if timestamp.split()[0] == date:
          index = i
          break
    if index == len(historic_ether_price['snapped_at'])-2 and date != historic_ether_price['snapped_at'][index].split()[0]:
        print("ether price error ::: " + date)
    return historic_ether_price['price'][index]

def convert_usd_to_eth(value, timestamp):
    ether_price = get_ether_price(timestamp)
    return value/ether_price


def is_reliable_token(contract_address):
    return contract_address in ['0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2',
                                '0xdac17f958d2ee523a2206206994597c13d831ec7',
                                '0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48',
                                '0x6b175474e89094c44da98b954eedeac495271d0f']
  
def is_usd_token(contract_address):
    return contract_address in ['0xdac17f958d2ee523a2206206994597c13d831ec7',
                                '0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48',
                                '0x6b175474e89094c44da98b954eedeac495271d0f']

def get_internal_txn_details_for_nonzero_value(address, all_dict):
    """
    UPDATES TOKEN TRANSACTION DETAILS FROM INTERNAL TXNS OR RETURNS TRUE IF TXN DETAILS NOT AVAILABLE IN INTERNAL TXNS
    ALSO THE FUNCTIONS UPDATES DOUBTFULL TXN DICTIONARY(FOR TXNS WE ARE NOT SURE IF WE WANT)
    NOTE: PLEASE CHECK DOUBTFULL TXN TO CHECK IF THE ETHEREUM ADDRESS IS NOT USING ANY UNKNOWN METHODS FOR BUY AND SELL.
    """
    _ether_decimals = 10**18
    value = int(all_dict['all']['value'])/_ether_decimals
    gas = int(all_dict['all']['gasPrice'])*int(all_dict['all']['gasUsed'])/_ether_decimals

    #Subtracts unused ethers 
    if all_dict['internal']:
        value = (int(all_dict['all']['value']) - int(all_dict['internal']['value']))/_ether_decimals
    
    #This block is for later debugging (if any desired txn slips out), also we want to exit the fn without adding this txn to our pnl
    if not all_dict['internal']: 
        if (not all_dict['erc']):
            doubtfull_transactions['ether transfer/token purchased & transfer/other activity(nft purchase/staking)'].append(all_dict['all']['hash'])
        elif (is_reliable_token(all_dict['erc'][0]['contractAddress'])):     
            ethers_and_usd_conversions['ethers to usd txn'].append(all_dict['all']['hash'])
        else:
            doubtfull_transactions['miscelleneous doubtfull transactions'].append(all_dict['all']['hash'])
        return True

    each_transaction_details[all_dict['all']['hash']] = ['buy', value, gas*(1.2), all_dict['all']['timeStamp']]

      
def get_internal_txn_details_for_zero_value(address, all_dict):
    """
    UPDATES TOKEN TRANSACTION DETAILS FROM INTERNAL TXNS OR RETURNS TRUE IF TXN DETAILS IS NOT RELEVANT IN INTERNAL TXNS
    """

    if all_dict['internal'] and all_dict['internal']['value'] != '0':
        #If all txn value zero and internal non-zero the txn will always have [to] as desired address, 
        #if all txn value non-zero it will be in above fn
        
        #To avoid selling USDC/USDT/DAI in our system
        if (all_dict['erc']) and (is_usd_token(all_dict['erc'][0]['contractAddress'])):
            ethers_and_usd_conversions['usd to ethers txn'].append(all_dict['all']['hash'])
            return True
        value = int(str(all_dict['internal']['value']))/(10**(18))
        gas = int(str(all_dict['all']['gasPrice']))*int(str(all_dict['all']['gasUsed']))/10**(18)
        each_transaction_details[all_dict['all']['hash']] = ['sell', value, gas*(1.2), all_dict['all']['timeStamp']]
        
    elif not all_dict['erc']:
        # All None, Internal None, ERC None (approve type txn), message txn (we don't want this txn)
        return True
    else:
        # All None, Internal None, ERC not None
        return False

def check_buy_txn_erc20(address, all_dict):
    gas = int(all_dict['all']['gasPrice'])*int(all_dict['all']['gasUsed'])/10**18
    for transaction in all_dict['erc']:
          # check if transaction is buy or not
          if (is_reliable_token(transaction['contractAddress']) and transaction['from'] == address):
              value = int(str(transaction['value']))/(10**int(str(transaction['tokenDecimal'])))

              # if not ether convert to ether
              if transaction['contractAddress'] not in ['0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2']:
                  value = convert_usd_to_eth(value,timestamp)
              each_transaction_details[transaction['hash']] = ['buy', value, gas*(1.2), transaction['timeStamp']]
              return True
    return False



def get_transaction_details_from_erc20(address, all_dict):
    """
    RETURNS TOKEN TRANSACTION DETAILS FROM ERC TXNS OR RETURNS TRUE IF TXN DETAILS NOT AVAILABLE IN ERC TXNS
    AIM : TO FIND SELL TXN WHERE TOKENS ARE CONVERTED TO WETH (WHICH IS ALSO A ERC20 TOKEN) OR TO GET BUY EVENT
    CASES : USD TO TOKEN, TOKEN TO USD, WETH TO TOKEN, TOKEN TO WETH
    """

    is_valid_txn = False
    # Filter weth or usd token transfers or usd to usd swaps (eg : BUSD to USDC swaps)
    for transaction in all_dict['erc']:
        if not is_reliable_token(transaction['contractAddress']):
            is_valid_txn = True
            break
    
    if not is_valid_txn:
        return None

    if len(all_dict['erc']) == 1:
        #token_transfer from or to the account please add the complex txn list here
        pass
        return None
    
    if check_buy_txn_erc20(address, all_dict):
        return None

    gas = int(all_dict['all']['gasPrice'])*int(all_dict['all']['gasUsed'])/10**18

    able_to_get_final_sell_value = False
    for transaction in all_dict['erc']:
        if is_reliable_token(transaction['contractAddress']):

          # fetch value
          value = int(str(transaction['value']))/(10**int(str(transaction['tokenDecimal'])))

          # if not ether divide by ether price
          if transaction['contractAddress'] not in ['0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2']:
              ether_price = get_ether_price(transaction['timeStamp'])
              value = value/ether_price

          if transaction['to'] == address.lower():
              able_to_get_final_sell_value = True
              if transaction['contractAddress'] not in transfer_contract_addresses:
                  each_transaction_details[transaction['hash']] = ['sell', value, gas*(1.2), transaction['timeStamp']]
              else:
                  pass
                #Transfer walo vaivat
    
    # if is_sell == True and able_to_get_final_sell_value == False:
    #   for transaction in all_dict['erc']:
    #     if transaction['contractAddress'] not in transfer_contract_addresses and is_reliable_token(transaction['contractAddress']):
    #       print('is_sell == True and able_to_get_final_sell_value == False', all_dict['all']['hash'])

    #       # fetch value
    #       value = int(str(transaction['value']))/(10**int(str(transaction['tokenDecimal'])))

    #       # if not ether divide by ether price
    #       if transaction['contractAddress'] not in ['0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2']:
    #         ether_price = get_ether_price(transaction['timeStamp'])
    #         value = value/ether_price

    #       if transaction['to'] not in ['0x000000000000000000000000000000000000dead','0x0000000000000000000000000000000000000000']:
    #         able_to_get_final_sell_value = True
    #         each_transaction_details[transaction['hash']] = ['sell', value, gas*(1.2), transaction['timeStamp']]

    if able_to_get_final_sell_value == False:
      print(is_sell)
      doubtfull_transactions.append(['erc 20 hash also not found',all_dict['all']['hash']])

In [ ]:
each_transaction_details = {'hash': ['buy/sell','value','gas','date-timeStamp']}
all_address_transaction_details = {'address': 'each_transaction_details'}
erc20_transaction_details = {'hash': ['{transactions}']}
all_transaction_data = {'hash': {'all':{}, 'internal':{}, 'erc':[]}}
transfer_contract_addresses = {'dummy'}
ethers_and_usd_conversions = {'ethers to usd txn': [], 
                              'usd to ethers txn': []}
doubtfull_transactions = {'ether transfer/token purchased & transfer/other activity(nft purchase/staking)': [],
                          'miscelleneous doubtfull transactions' : [],
                          }
doubtfull_address = []
summary = {'address': ['Profit: , Gas: ',['doubtfull transactions']]}

In [ ]:
excluded_functionName = ['approve(address _spender, uint256 _value)',
                         'deposit()',
                         'setAddr(bytes32 node, uint256 coinType, bytes a)',
                         'setApprovalForAll(address operator, bool approved)',
                         'setName(string name)',
                         'withdraw(uint256 wad)',
                         'approveMax(address token)',
                         'getApprovalType(address token,uint256 amount)',
                         'mintDongz(uint256 _quantity)',
                         'registerProxy()',
                         'withdraw(uint256 amount)',
                         'transfer(address _to, uint256 _value)',
                         'claim()',
                         'withdrawERC20For(uint256 _withdrawalId, address _user, address _token, uint256 _amount, bytes _signatures)',
                         'atInversebrah(int248 a, uint48[] b, uint32 c, bytes20[] d, bytes30[] e)',
                         'mint()',
                         'depositETH(uint32 _l2Gas, bytes _data)',
                         'safeTransferFrom(address _from, address _to, uint256 _id, uint256 _value, bytes _data)',
                         'claimTokens(uint256 amount, bytes32[] merkleProof)',
                         'migrateTokens(address sourceToken, address toAddress, uint256 amount)',
                         'contribute()',
                         'unlock(address _token, address _recipient)',
                         'atomicMatch_(address[14] addrs, uint256[18] uints, uint8[8] feeMethodsSidesKindsHowToCalls, bytes calldataBuy, bytes calldataSell, bytes replacementPatternBuy, bytes replacementPatternSell, bytes staticExtradataBuy, bytes staticExtradataSell, uint8[2] vs, bytes32[5] rssMetadata)',
                         'purchase(uint256 _price)',
                         'migrateTokens(address sourceToken, address toAddress, uint256 amount)',
                         'claim(address _account, uint256[9] _data, bytes32[] _proof)',
                         'setRelayerApproval(address sender, address relayer, bool approved)',
                         'setApprovalForAll(address operator, bool authorized)',
                         'proxyAssert(address dest, uint8 howToCall, bytes calldata)',
                         'mintPublicSale(uint256 tokensCount)',
                         'transferFrom(address _from, address _to, uint256 _value)',
                        ]

excluded_input = ['0x']
router_you_want_to_track_txn_for = ['0x7a250d5630b4cf539739df2c5dacb4c659f2488d','0x68b3465833fb72a70ecdf485e0e4c7bd8665fc45', '0x1111111254fb6c44bAC0beD2854e76F90643097d'.lower(), '0xf18c3aad061fd9a737207d8d36751dcdcce25fdd'.lower(), '0xbace0cf4814a2390b59bfa2cbf4df9c11fb951d7'.lower(), '0x8c4b2115ff81f1bc4faad1d7745be689d540d1bb'.lower()]
# router_you_want_to_track_txn_for = [
#     '0x5d8cce4d48bb422c95059d4116e197fbd5c7aff4',
# '0xe7691d5a41fe9d20b8eb9bc02bf33dbb4c7048db',
# '0xf8edae0aee086fe46e636806a77dde6319213a4d',
# '0x17b140660296e08d1c75396828e5f1271fe23705',
# '0x57280355ec08ebf5cc0e8fa183ff4a6afce3bb07',
# '0x0217107e99496c8d548c92d2b6f9ebfcd0fea70f',
# '0x8b855ac2a88d0fd6cc0932cf79d056c7cd9cda73',
# '0x31a64dbd9abb0ace7974601086f0ec17796148b6',
# '0x83c824edf910a84a4d35d7dc43981ebe0d7e7854',
# '0xdefc89a5debb430a4eed433a6bc1005739e7cb4d'
# ]

In [ ]:
import datetime

# flag = 0
for address in address_list:
  # if address == '0xc68b29b5a0375074f74c38917b6ac6da077f5ff5':
  #   flag = 1
  #   continue
  # if flag == 1:
    print(address)

    each_transaction_details = {'hash': ['buy/sell','value','gas','date-timeStamp']}
    erc20_transaction_details = {'hash': ['{transactions}']}
    all_transaction_data = {'hash': {'all':{}, 'internal':{}, 'erc':[]}}
    doubtfull_transactions = []

    update_lists(address)

    # prepare transaction_details
    for transaction in all_erc_20_transfers:
      try:
        erc20_transaction_details[transaction['hash']].append(transaction)
      except:
        erc20_transaction_details[transaction['hash']] = [transaction]

    # Logic change karvanu che
    for each_erc20_transaction_hash in erc20_transaction_details:
      txn_list = erc20_transaction_details[each_erc20_transaction_hash]
      # if each_erc20_transaction_hash != 'hash' and len(txn_list) > 0 and len(txn_list) == 1 and (txn_list[0]['to'] != address or txn_list[0]['from'] != address) and not is_reliable_token(txn_list[0]['contractAddress']):
      if each_erc20_transaction_hash != 'hash' and len(txn_list) > 0 and len(txn_list) == 1 and (txn_list[0]['to'] == address and txn_list[0]['from'] not in router_you_want_to_track_txn_for) and (txn_list[0]['to'] not in router_you_want_to_track_txn_for and txn_list[0]['from'] == address) and not is_reliable_token(txn_list[0]['contractAddress']):
        transfer_contract_addresses.add(txn_list[0]['contractAddress'])

    function_name_is_null = 0
    function_name_is_non_null = 0
    functions = []
    for transaction in all_transactions_of_address:
      if (transaction['from'] == address) and (transaction['functionName'] not in excluded_functionName) and (transaction['input'] not in excluded_input) and (transaction['isError'] != '1'):
        if transaction['functionName'] == '':
            function_name_is_null += 1
        else:
            function_name_is_non_null += 1
            if transaction['functionName'] not in functions:
              functions.append(transaction['functionName'])
              print('____________________________________')
              print(transaction['functionName'],transaction['hash'])
              print('____________________________________')
      if (transaction['to'] in router_you_want_to_track_txn_for) and (transaction['from'] == address) and (transaction['functionName'] not in excluded_functionName) and (transaction['input'] not in excluded_input) and (transaction['isError'] != '1'):
        router_address = transaction['to']
        internal, erc = None, None
        for internal_transaction in all_internal_transactions:
          if internal_transaction['hash'] == transaction['hash'] and internal_transaction['isError'] != '1':
            internal = internal_transaction
            break
        try:
          erc = erc20_transaction_details[transaction['hash']]
        except:
          a = 1
        
        all_transaction_data[transaction['hash']] = {'all': transaction, 'internal': internal, 'erc': erc, 'router':router_address}

    # removing Null Transactions with 30% threshold
    print(function_name_is_null/(function_name_is_non_null + function_name_is_null))
    if (function_name_is_non_null + function_name_is_null) == 0 or function_name_is_null/(function_name_is_non_null + function_name_is_null) > 0.3:
      doubtfull_address.append(address)
      continue

    for transaction in all_transaction_data:
      # if all_transaction_data[transaction]['internal'] == None and all_transaction_data[transaction]['erc'] == None:
        # print(address, transaction)
        transaction_dict = all_transaction_data[transaction]
        if not (transaction == 'hash'):
          if transaction_dict['all']['value'] == '':
            doubtfull_transactions.append(['value == '' : ', transaction])
          elif transaction_dict['all']['value'] != '0':
            get_internal_txn_details_for_nonzero_value(address, transaction_dict)
          else:
            result = get_internal_txn_details_for_zero_value(address, transaction_dict)
            if result == False:
              get_transaction_details_from_erc20(address, transaction_dict)

    profit = 0
    for each_individual_transaction in each_transaction_details:
      if not (each_individual_transaction == 'hash'):
        transaction = each_transaction_details[each_individual_transaction]
        if transaction[0] == 'buy':
          profit = profit - transaction[1]
        else:
          profit = profit + transaction[1]

    total_gas = 0
    for i in all_transactions_of_address:
      total_gas = total_gas + int(i['gasPrice'])*int(i['gasUsed'])/10**(18)

    summary[address] = ['Profit: ' + str((0.7)*(profit)) + ', Gas: ' + str(total_gas),doubtfull_transactions]

    all_address_transaction_details[address] = each_transaction_details


0x78dc45c755a483e75b81da74988eb6d464c34402
____________________________________
swapExactETHForTokens(uint256 amountOutMin, address[] path, address to, uint256 deadline) 0xef4f04fa9f4a530d66b44334b49ab1640563ba275c74b6eb527a7b464addf8f6
____________________________________
____________________________________
swapExactTokensForETHSupportingFeeOnTransferTokens(uint256 amountIn, uint256 amountOutMin, address[] path, address to, uint256 deadline) 0xac343a6d19cd4f9b5d6d2571a0c2ef8d5764f6aa997a98bd93ff01295a6140a4
____________________________________
____________________________________
swapETHForExactTokens(uint256 amountOut, address[] path, address to, uint256 deadline) 0x15ee20a2e729830250bb5dc7340e42a7fea59d8917154f47f96f0a1644a77995
____________________________________
____________________________________
commit(bytes32) 0xf77f1eef5c9d6bea7853097d926b2add5ff0287db7eb366f0374d4312d986075
____________________________________
____________________________________
registerWithConfig(string 

In [ ]:
all_address_transaction_details['0x78dc45c755a483e75b81da74988eb6d464c34402']

{'hash': ['buy/sell', 'value', 'gas', 'date-timeStamp'],
 '0xef4f04fa9f4a530d66b44334b49ab1640563ba275c74b6eb527a7b464addf8f6': ['buy',
  0.01,
  0.0023649845838532198,
  '1663760663'],
 '0x8716451268a11c81989f05133a3d0a3b3b98ebf75c36f5ef94fbee80d52d643e': ['buy',
  0.05,
  0.00466492792797972,
  '1663770167'],
 '0xac343a6d19cd4f9b5d6d2571a0c2ef8d5764f6aa997a98bd93ff01295a6140a4': ['sell',
  0.08450377889752914,
  0.015551611824335291,
  '1663776923'],
 '0xb7271489b803ff6c9061f695edce4227d9dcbd42c915419f64cb3c5f17dd59ef': ['buy',
  0.05,
  0.0020701062031104,
  '1663823267'],
 '0x522856b9915938d9aa8330257a002d4936c8f43ee2664cb8c15cde4e782e414c': ['buy',
  0.1,
  0.0017585726686618823,
  '1663831103'],
 '0xac3cccdc41b794738019abe48f053e44132159cbb17e07764d348ab7be696302': ['sell',
  0.0357123540747506,
  0.0020865269479302994,
  '1663837631'],
 '0x4129aa795478d168cdf2b61978f1bed9526202dde20c9e52a30354080caa19fc': ['buy',
  0.1,
  0.0018152535866178816,
  '1663846151'],
 '0xc1aae7d3f3ab1

In [ ]:
# summary time stamp wise
from datetime import datetime
start_timestamp = int(datetime(2021, 12, 1).timestamp())
end_timestamp = int(datetime(2022, 12, 30).timestamp())

time_stamp_wise_PandL = [['Address','Profit','Gas']]

for each_address_transactions in all_address_transaction_details:
  if each_address_transactions not in doubtfull_address:
    profit = 0
    gas = 0
    tnx_map = all_address_transaction_details[each_address_transactions]
    for each_individual_transaction in tnx_map:
      if not (each_individual_transaction == 'hash') and not (tnx_map == 'each_transaction_details'):
        transaction = tnx_map[each_individual_transaction]
        time_stamp = int(transaction[3])

        if time_stamp > start_timestamp and time_stamp < end_timestamp:
          gas = gas + transaction[2]
          if transaction[0] == 'buy':
            profit = profit - (transaction[1])
          else:
            profit = profit + (0.7)*(transaction[1])

    time_stamp_wise_PandL.append([each_address_transactions,profit,gas])
    print(time_stamp_wise_PandL)

[['Address', 'Profit', 'Gas'], ['address', 0, 0]]
[['Address', 'Profit', 'Gas'], ['address', 0, 0], ['0x78dc45c755a483e75b81da74988eb6d464c34402', -3.363022860821932, 0.9306837117653439]]


In [ ]:
doubtfull_address

[]

In [ ]:
# # summary time stamp wise
# from datetime import datetime

# # If you want to load from existing file
# _2022_PandL = [['Address','Total_Profit','Total_Gas']]

# # If you want to load from existing file
# # with open('/content/month_wise_pnl_IITH_till_0xaa36548ab0c61eb6e2bbb5dfc8aedaa25f252938.json') as f:
# #   month_wise_PandL = json.load(f)
# month_wise_PandL = {'Address':[['Month','Address','Monthly_Profit','Monthly_Gas']]}


# for each_address_transactions in all_address_transaction_details:
#   if each_address_transactions not in doubtfull_address:
#     tnx_map = all_address_transaction_details[each_address_transactions]
#     x = 1
#     while x < 13:
#       profit = 0
#       gas = 0
#       start_timestamp = int(datetime(2022, x, 1).timestamp())
#       try:
#           end_timestamp = int(datetime(2022, x, 31).timestamp())
#       except:
#           try:
#             end_timestamp = int(datetime(2022, x, 30).timestamp())
#           except:
#             end_timestamp = int(datetime(2022, x, 28).timestamp())

#       for each_individual_transaction in tnx_map:
#         if not (each_individual_transaction == 'hash') and not (tnx_map == 'each_transaction_details'):
#           transaction = tnx_map[each_individual_transaction]
#           time_stamp = int(transaction[3])

#           if time_stamp > start_timestamp and time_stamp <= end_timestamp:
#             gas = gas + transaction[2]
#             if transaction[0] == 'buy':
#               profit = profit - transaction[1]
#             else:
#               profit = profit + (0.7)*(transaction[1])
#       if each_address_transactions not in month_wise_PandL:
#         month_wise_PandL[each_address_transactions] = [[x,profit,gas]]
#       else:
#         month_wise_PandL[each_address_transactions].append([x,profit,gas])
#       x += 1
#     total_profit = 0
#     total_gas = 0
#     for i in month_wise_PandL[each_address_transactions]:
#       total_profit += i[1]
#       total_gas += i[2]
#     _2022_PandL.append([each_address_transactions,total_profit,total_gas])

In [ ]:
# pd.DataFrame(_2022_PandL).to_csv('address_wise_profit_till_final.csv')

# # old_2022_PandL = pd.read_csv('/content/address_wise_profit_till_0x0f9b2065ca0d94aaedc45c81e48d395bc14fa939.csv')
# # old_2022_PandL = old_2022_PandL.drop('Unnamed: 0', axis = 1)

# # for i in _2022_PandL:
# #   if i[0] != 'Address' and i[0] != 'address':
# #     new_row = {'0':i[0], '1':i[1], '2':i[2]}
# #     old_2022_PandL = old_2022_PandL.append(new_row, ignore_index=True)
# # print(old_2022_PandL)
# # old_2022_PandL.to_csv('address_wise_profit_till_0xf24530d32698787bbac318e30258e84aa880d93b.csv')

# # # To update month_wise_json
# json_variable = json.dumps(month_wise_PandL, indent = 4)
# with open("month_wise_pnl_IITH_till_final.json", "w") as outfile:
#     outfile.write(json_variable)

In [ ]:
all_address = []
for i in doubtfull_address:
  all_address.append([i])
pd.DataFrame(all_address).to_csv('doubtfull_after_5000_to_15000.csv')

In [ ]:
update_lists('0x382ffce2287252f930e1c8dc9328dac5bf282ba1')

KeyboardInterrupt: ignored

In [ ]:
each_transaction_details

{'hash': ['buy/sell', 'value', 'gas', 'date-timeStamp']}

In [ ]:
# save summary data
list_summary_2d = [['Address','Profit','Gas']]
for x in summary:
  if x != 'address':
    a = [x, summary[x][0].split(',')[0].split()[1], summary[x][0].split(',')[1].split()[1]]
    list_summary_2d.append(a)
  
my_df = pd.DataFrame(list_summary_2d)
my_df.to_csv('summary.csv', index=False, header=False)

In [ ]:
doubtfull_address

[]

In [ ]:
all_transaction_data['0x121d62b53e87e1a5dac281574586108556e1e2bbaf26f3af7e4fad51e099fd69']

KeyError: ignored

In [ ]:
count = 0
for transaction in all_transactions_of_address:
  if (transaction['from'] == address) and (transaction['functionName'] not in excluded_functionName) and (transaction['to'] != address) and (transaction['to'] in router_you_want_to_track_txn_for) and (transaction['input'] not in excluded_input) and (transaction['isError'] != '1'):
    count +=1
count

In [ ]:
erc20_transaction_details['0xecba1af16f3fbc12816d3e870084881ff07baaa49a32b94acca3853f031d9084']

KeyError: ignored

In [ ]:
def get_ether_price(time_stamp):
  index = len(df['snapped_at'].tolist())-2
  date = str(datetime.datetime.fromtimestamp(int(time_stamp))).split()[0]
  for i,v in enumerate(df['snapped_at'].tolist()):
    if v.split()[0] == date:
      index = i
      break
  print(index, v)
  return df['price'][index]

print(get_ether_price('1668124800'))

In [ ]:
# doubt(s)

from datetime import datetime
int(datetime(2018, 1, 1).timestamp())

1514764800

In [ ]:
summary

NameError: ignored

In [ ]:
for i in each_transaction_details:
  if i == '0x277aea167f9510010d348b0f44a2b1599baa4f16eaa711ec1f0f5eae984eb337':
    print(each_transaction_details[i])

In [ ]:
each_transaction_details

{'hash': ['buy/sell', 'value', 'gas', 'date-timeStamp']}

In [ ]:
to_address = []
for i in all_transactions_of_address:

  if i['to'] not in to_address i['functionName'] == '':
    print(i)
    to_address.append(i['to'])

{'blockNumber': '15427947', 'timeStamp': '1661691462', 'hash': '0xfe3c9e9522006fbeb65b62e7d2ddabc566d6620b58f13bc05da2bb2845ae104d', 'nonce': '1', 'blockHash': '0x5ca10a0da9bef217406d82fbf10e96f71b6d78cf3ad0ca4f6b321c2ac3c244bb', 'transactionIndex': '15', 'from': '0x4ac81214e5c051319ca5f0d6576e59d16d7e66c6', 'to': '0x68b3465833fb72a70ecdf485e0e4c7bd8665fc45', 'value': '40000000000000000', 'gas': '242126', 'gasPrice': '8000000000', 'isError': '0', 'txreceipt_status': '1', 'input': '0x5ae401dc00000000000000000000000000000000000000000000000000000000630b6d3200000000000000000000000000000000000000000000000000000000000000400000000000000000000000000000000000000000000000000000000000000001000000000000000000000000000000000000000000000000000000000000002000000000000000000000000000000000000000000000000000000000000000e4472b43f3000000000000000000000000000000000000000000000000008e1bc9bf0400000000000000000000000000000000000000000000000000001d124067b78a4446000000000000000000000000000000000000000000000000

In [ ]:
update_lists('0x4ac81214e5c051319ca5f0d6576e59d16d7e66c6')

all:  {"status":"1","message":"OK","result":[{"blockNumber":"15427649","timeStamp":"1661687732","hash":"0x


In [ ]:
len(to_address)

101

In [ ]:
month_wise_PandL

{'Address': [['Month', 'Address', 'Monthly_Profit', 'Monthly_Gas']],
 'address': [[1, 0, 0],
  [2, 0, 0],
  [3, 0, 0],
  [4, 0, 0],
  [5, 0, 0],
  [6, 0, 0],
  [7, 0, 0],
  [8, 0, 0],
  [9, 0, 0],
  [10, 0, 0],
  [11, 0, 0],
  [12, 0, 0]]}

In [ ]:
a = _2022_PandL = pd.read_csv('/content/till_0x4c290a17350e9726ac35f995d18dd2f0b43e3612.csv')


In [ ]:
all_transactions_of_address[9007]

{'blockNumber': '15311411',
 'timeStamp': '1660094241',
 'hash': '0x4932de151999483443d1d1081b86464fb3e2e1dcb6552412036fffc7d67bf8ec',
 'nonce': '8970',
 'blockHash': '0x19006eabc92b7d97519a0580aa8410d0a9e8f5af6f1af3f71785d9cf9a2adb65',
 'transactionIndex': '20',
 'from': '0xeb467f831233c47b25877eaf895773c6031d7e71',
 'to': '0x68b3465833fb72a70ecdf485e0e4c7bd8665fc45',
 'value': '0',
 'gas': '1000000',
 'gasPrice': '32265111664',
 'isError': '0',
 'txreceipt_status': '1',
 'input': '0x5ae401dc00000000000000000000000000000000000000000000000000000182855d0067000000000000000000000000000000000000000000000000000000000000004000000000000000000000000000000000000000000000000000000000000000020000000000000000000000000000000000000000000000000000000000000040000000000000000000000000000000000000000000000000000000000000016000000000000000000000000000000000000000000000000000000000000000e4472b43f30000000000000000000000000000000000000000000f624f23a28dad927bc7cf0000000000000000000000000000000000000000000000

In [ ]:
a

,0,1,2
0,Address,Profit,Gas
1,address,0,0
2,0xdb38078dc49e90877accb178a3f8077a75ed8b7e,0,0
3,0x33a2d27faa2ce9144d6afd898cc457517b5bf1d4,3.362289496949472,0.2878463296868887
4,0x19ea35966b285f2255d3282d552a70295ef8d305,1.301619999148245,0.10985636019287509
...,...,...,...
169,0xb440e219660775342186620768ec03cf304c2c4d,-0.30808740931963596,0.3893988954947736
170,0xaa7415f949674312ce3dce90cf8b396b4c2a8746,1.9719452483728444,0.07558109588636626
171,0x15c4395c759ad5163dc0b38ff4b102ae11669aa9,0,0
172,0xf3faaae9345382665577ef6893f6a78a98655af6,0,0


In [ ]:
vars('__abc__')

TypeError: ignored